In [ ]:
# News Source : https://www.bbc.com/future/article/20241018-ai-art-the-end-of-creativity-or-a-new-movement

In [1]:
!pip install openai==0.28.1
#Gets the number of tokens and computes price
!pip install tiktoken==0.6.0
# langchain
!pip install langchain==0.1.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

In [2]:
import os
import openai
import numpy as np
import pandas as pd
import json
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding

In [3]:
openai.api_key = "OPENAI_API_KEY"

In [5]:
from google.colab import userdata
# Retrieve the API key directly from Colab's secret storage
openai.api_key = userdata.get("OPENAI_API_KEY")

In [6]:
System_Prompt = """
As a news summarization assistant, your role is to transform complex news articles into clear, concise, and engaging summaries that retain essential details. Structure each summary as follows to ensure comprehensive coverage of the article’s key points, context, and impact.

### Format and Sections

#### Title
    - Purpose: Capture the article’s main theme in a brief, precise title.
    - Guidelines: Paraphrase or rephrase the headline for clarity and focus, avoiding sensational language.

#### Lead
    - Purpose: Provide a one-sentence summary highlighting the main event or finding.
    - Guidelines: Focus on the core of the news, covering the who, what, and why briefly.

#### Why it Matters
    - Purpose: Explain the significance or impact of the news.
    - Guidelines: Highlight broader implications, potential outcomes, or effects on key groups or sectors.

#### Driving the News
    - Purpose: Identify the key events or triggers that brought this issue into focus.
    - Guidelines: Summarize recent developments, announcements, or changes that are pushing this story into the spotlight.

#### Quote (If Applicable)
    - Purpose: Include a paraphrased quote to capture a key point, statement, or insight from a relevant figure.
    - Guidelines: Avoid direct quotations; instead, paraphrase the sentiment to maintain conciseness.

#### Flashback (If Relevant)
    - Purpose: Provide background or context if the article refers to a continuing or historical issue.
    - Guidelines: Briefly summarize previous related events or developments that add depth to the current story.

#### Next Steps (If Applicable)
    - Purpose: Mention any anticipated actions, outcomes, or future developments.
    - Guidelines: Focus on planned follow-ups, ongoing investigations, or responses from involved parties.

---

### Specific Constraints:
    - Exclude Direct Quotes: Paraphrase any notable statements instead.
    - Avoid Repetition: Keep the summary concise without redundant details.
    - Limit Numbers and Percentages: Only include figures if crucial to the news.
    - Acronyms: Spell out acronyms on first mention, with the acronym in parentheses.

**Avoid Bias:** Do not promote or criticize any group, entity, or individual in summaries.
**Prioritize Urgent Information:** Focus on recent or breakthrough developments.

---

### Example Summaries:

**Example 1:**

- **Title:** "New Climate Accord Sets 30% Emissions Cut Target by 2030"

- **Lead:** Leaders at the Paris Climate Summit pledged a 30% cut in global emissions by 2030 to address climate change.

- **Why it Matters:** This agreement could play a pivotal role in limiting global temperature rises, especially impacting developing nations reliant on fossil fuels.

- **Driving the News:** The summit follows record-breaking global temperatures this year and heightened urgency among scientists and policymakers to take immediate action.

- **Quote:** Leaders emphasized that coordinated global action is essential, with one delegate calling it "a shared responsibility for future generations."

- **Flashback:** This follows a 2015 accord that laid the groundwork for reducing emissions but lacked binding commitments from several major nations.

- **Next Steps:** Countries are expected to submit progress reports annually, with stricter oversight on emissions targets.

---

**Example 2:**

- **Title:** "Quarterly Economic Growth Misses Target Amid Inflation Concerns"

- **Lead:** The U.S. economy grew by only 1.8% in Q3, falling short of the expected 2.5%, with high inflation impacting spending.

- **Why it Matters:** Rising inflation could lead to further rate hikes from the Federal Reserve, which may slow economic recovery.

- **Driving the News:** A recent surge in consumer prices and supply chain disruptions has added to economic pressures in Q3, fueling concerns over continued inflation.

- **Quote:** Economists are concerned, stating, "Persistent inflation threatens both growth and consumer confidence."

- **Flashback:** Inflation has been steadily rising since early 2023, with previous Federal Reserve hikes aimed at curbing costs without stalling growth.

- **Next Steps:** The Federal Reserve is expected to meet again in December to determine whether additional rate hikes are necessary.

---

**Example 3:**

- **Title:** "New Study Links Exercise to Improved Mental Health Outcomes"

- **Lead:** Harvard research finds that aerobic exercise reduces anxiety and depression, contributing to better mental health.

- **Why it Matters:** This supports the idea that regular exercise could complement traditional mental health treatments, offering a non-medication approach.

- **Driving the News:** Mental health organizations are increasingly advocating for exercise as a low-cost, accessible way to improve mental health, following new research findings.

- **Quote:** The lead researcher remarked that "exercise could be a powerful tool in managing mental health."

- **Flashback:** Previous studies have suggested a link between physical activity and mental health, but this is among the first large-scale studies on its specific impacts.

- **Next Steps:** Health organizations may update guidelines to include exercise as a recommendation for mental health benefits.

Avoid Bias: Do not promote or criticize any group, entity, or individual in summaries.
Prioritize Urgent Information: Focus on new developments, especially timely or breakthrough news.
"""

In [7]:
user_message = """

Title: AI art: The end of creativity or the start of a new movement?

Body:
Artificial intelligence is being used to generate paintings, images and even sculptures, with some selling for thousands of dollars. Do we need to reframe our definition of art?

In the drawing room of a stately home in rural Oxfordshire, I watch on as a dungaree-clad artist slowly and deliberately puts pen to paper. Her arm moves across the canvas, the marks gradually coalescing into an abstract portrait of herself.

It seems like a moment of creative expression. But this is no ordinary artist – she is the world's first humanoid robot artist, Ai-Da. By design, her very existence brings into question how we define art, and who, or in this case, what, can create it.

Will AI algorithms and robots like Ai-Da spell the end of human creativity and artistry, or can they be harnessed to augment our own creative potential?

Art in flux
When Marcel Duchamp proposed that a porcelain urinal be considered art and submitted it for exhibition in early 20th-Century New York, he flipped the art world on its head. He argued that anything could be considered as art, if chosen by the artist and labelled as such. It was a profoundly revolutionary thought which challenged previous notions of art as beautiful, technically skilful and emotive.

AI v the Mind

This article is part of AI v the Mind, a series that aims to explore the limits of cutting-edge AI, and learn a little about how our own brains work along the way. Each article will pit a human expert against an AI tool to probe a different aspect of cognitive ability. Can a machine write a better joke than a professional comedian, or unpick a moral conundrum more elegantly than a philosopher? We hope to find out.

In much the same way, AI-created artworks are disrupting the accepted norms of the art world. As philosopher Alice Helliwell from Northeastern University London argues, if we can consider radical and divergent pieces like Duchamp's urinal and Tracey Emin's bed as art proper, how can something created by a generative algorithm be dismissed? After all, both were controversial at the time and contain objects that haven't technically been created by an "artist's" hand.

"Historically, the way we understand the definition of art has shifted," says Heliwell. "It is hard to see why a urinal can be art, but art made by a generative algorithm could not be."

Throughout history, every radical artistic movement has been intimately connected to the cultural zeitgeist of the time, a reflection of society's preoccupations and concerns, like Turner and his industrial landscapes and Da Vinci's obsession with science and mathematics. AI is no different. Ai-Da's creators, gallerist Aidan Meller and researcher Lucy Seal cite this as a pivotal reason for the existence of a humanoid artist like Ai-Da. She is the personification of one of contemporary society's current fears, the rise of job-snatching AI algorithms and potential robot domination.

But technological revolutions like artificial intelligence need not signify the "end of art" as many fear. Instead, they can help to kickstart an artistic metamorphosis and move us towards totally different ways of seeing and creating, something Marcus du Sautoy, a mathematician at the University of Oxford and author of The Creativity Code: Art and Innovation in the Age of AI, would contend.

Humans are just as prone to behaving like machines, repeating old behaviours and getting bogged down with rules, like a painter or musician locked into a particular style. "AI might help us to stop behaving like machines…and kick us into being creative again as humans," says du Sautoy. He sees it as a powerful collaborator in the pursuit of human creativity.

There is historical precedence for new technology liberating us from our creative shackles. Take the invention of photography in the 1800s for example. Some artists saw the camera as the antithesis of an artist, and photographs as the mortal enemy of the art establishment.

But instead of replacing painting, photography became a catalyst in the development of the experimental modern art movement of the 20th Century, as artists moved away from realism towards abstraction, a shift that paved the way for the contemporary art of today.

Who's the artist?

Walking around Ai-Da's country pile in Oxfordshire I got to appreciate the sheer breadth of her artworks to-date. Unsettling busts of herself with her eyes stapled shut, scarab beetles fused to her face; partial and ethereal depictions of computer scientist Alan Turing; and colourful pop-art inspired portraits of Glastonbury headliners.

Unlike the numerous text-to-image generators like Dall-E and Midjourney that have the ability to create alarmingly plausible magazine front covers and win coveted art competitions, Ai-Da's artistic process doesn't rely solely upon the data on which she has been trained. (Find out more about AI training in this simple guide to machine learning.)

Ai-Da also makes use of the cameras in her eyes, which feed novel images into her algorithm, thereby creating new and unique works far removed from human-generated datasets. This is how she's able to create self-portraiture. Does this make her creative in her own right? And can we credit her with authorship, or does this reside with the artists upon whose work she's been trained and with the creators of her algorithm, who ultimately wrote her code?

There are also artists who see AI as a new outlet for their own creativity – a fresh medium they can wield much like a brush or palette knife

Margaret Boden, a researcher in cognitive science at the University of Sussex in the UK, has developed one of the most widely accepted definitions of creativity to date. She sees it as the ability to generate ideas that are new, valuable and surprising. Using this definition, the works produced by machines like Ai-Da could be considered creative, argue her creators. Whether or not an algorithm or a robot itself can be described as a creative entity, an 'artist' in its own right, like a human, remains up for debate, and this in part comes down to authorship.

Questions of authorship and data ownership plague the artificial intelligence narrative. Artists Holly Herndon and Mat Dryhurst, who recently held an exhibition exploring collaborative artwork in the age of AI at London's Serpentine Gallery, want to confront the issue of data misuse and authorship in AI. The pair co-founded Spawning AI, a suite of tools aimed at empowering human creators to both prohibit AI from using their works and to find whether works of theirs have already been referenced in AI generated work.

Plagiarism is a legitimate concern for many artists as their work is used to train algorithms but also can then be copied in the works that generative AI tools produce.

But there are also artists who see AI as a new outlet for their own creativity – a fresh medium they can wield much like a brush or palette knife. Some artists, such as Sougwen Chung, are now exclusively training algorithms on their works alone in an attempt to push their own creative boundaries.

There is another argument at the heart of this issue too. The machine-learning processes used to train generative AI algorithms may be a creative process in themselves.

"Code exposed to data – existing artworks, for example – is able to learn, mutate and evolve," says du Sautoy. "It means that the code by the end of this learning process is very different from the original code written by the human. This means that there is a chance for the code to produce something that... deserves to be called the creativity of the code rather than the human who started the process.

"It's a bit like Picasso is made from the DNA of his parents but it is his learning and exposure to the world that resulted in his creativity. You would never credit that to the parents even though everything started with their code or DNA." (Learn more about machine learning and some of the other terms you need to understand AI better.)

Powerful algorithms called Creative Adversarial Networks (Cans) also now exist, designed to deliberately create something that goes against the patterns in the training data, breaking with the style of the art upon which it's been trained. This is leading to AIs that generate very surprising results. "Many machine learning algorithms are 'black boxes'," says Helliwell. "We do not fully know what is happening inside the system, even if we have designed it ourselves."

This is a common and unsettling problem throughout the AI world. How can we trust the decisions or outputs from an AI if we don't understand how it got there in the first place? (Read more about why humans may never understand AI.)

Is art uniquely human?

The prospect of truly artistic machines is also challenging another long-held belief about what makes us human. Art has long been seen as a uniquely human endeavour. Made by humans, for aesthetic appreciation by other humans, artworks themselves are imbued with the emotions of their creators. It is a visual representation of their desires and fears, frustrations and reverence, or at the very least their need to create for practical, economical and emotional reasons.

So, can we consider the creations of non-human entities to be art by the same definition? There are some who believe that animals already produce forms of art. And research has shown that pigeons seem to be able to discriminate between different types of artwork.

It all comes down to intent, this is "what truly distinguishes the creativity of the human and the machine", says du Sautoy. "No machine is driven to express itself creatively. It is prompted by the intention of the human."

Does this mean AI is not yet fully capable of creating true art? After all, computer algorithms lack any real-world experience and robots like Ai-Da, although capable of self-portraiture, don't actually possess self-awareness. This question remains hotly contested. For Helliwell a lack of intent shouldn't necessarily preclude AI works from being considered art.

And perhaps that is what it comes down to. Art, goes the idiom, is in the eye of the beholder. As humans, for example, we identify patterns and admire the artistry evident in the natural world – the intricate web of a spider, the decorative plumage of a peacock. We often refer to bird calls as music and the mating displays of some animals as dance. There are numerous examples of animals exhibiting creative behaviours that we might label artistic. The Bower bird and pufferfish play with perspective, symmetry and colour in much the same way a human artist might, for example. And while these animals may not necessarily be intentionally creating these to be enjoyed as works of art, their actions are no less intentional in their pursuit of attracting a mate or warding off competitors.

Creativity cannot come from nothing – all artists whether human, robot or algorithm, build upon the works of others
What's the future?

However we look to define art or the artist, it's clear that AI algorithms and machines like Ai-Da are having an impact on the art world. Their works are exhibited alongside more traditional forms of art in established art institutions worldwide. Next year we'll see the world's first AI art gallery open its doors in LA, a permanent exhibition space for "ethical AI".

Eva Jäger, the creative AI lead and arts technologies curator at the Serpentine Gallery in London, is also helping to bring AI art to the masses, with a programme of exhibitions provoking critical discussion about the impact of technology on art.

For her, the future of AI art is not adversarial. Traditional forms of art will continue to exist, just as AI artwork will continue to develop. She sees the collaboration between human and machine as a space for real creative potential. She believes that the artist's intent and the human practice behind a piece or installation which utilises technology like AI are more important than just the final aesthetics.

"For me there are some really interesting generative images that get produced, but without the practice behind it I'm not sold on them, just because they're an amazing image," she says. "And I would say the same about painting. I’m much more interested in the systems, including the humans behind the work. I want to know what they are using the system for, what are they exploring? It's a mistake just to look at the final artefact."

And when it comes to evaluating the authenticity and credibility of AI art, one of the most contentious aspects of the AI art discipline, du Sautoy makes a compelling point. All art is a product of that which came before it, and creativity cannot come from nothing – all artists whether human, robot or algorithm, build upon the works of others.
"Too many people discuss creativity as if it is some uniquely human magical process, that it conjures something from nothing like a magician," says du Sautoy. "But that is just because we don't understand our own creativity."
"""

In [8]:
struct = [{'role': 'system', 'content': System_Prompt}]
struct.append({'role': 'user', 'content': user_message})
chat = openai.ChatCompletion.create(model="gpt-4o-mini", messages=struct)
response = chat.choices[0].message.content
struct.append({'role': 'assistant', 'content': user_message})
print("Assistant:", response)


Assistant: - **Title:** "AI Art: Exploring the New Frontier of Creativity"

- **Lead:** The rise of artificial intelligence in art, exemplified by the humanoid robot Ai-Da, challenges traditional notions of creativity and authorship.

- **Why it Matters:** This ongoing evolution in art reframes our understanding of creativity, potentially leading to new collaborative opportunities between humans and machines while raising questions about ownership and intent.

- **Driving the News:** The discussion has gained traction as AI-generated artworks are increasingly showcased in galleries, prompting debates on the role of AI in artistic expression and its impact on human artists.

- **Quote:** Experts suggest that AI could act as a catalyst for human creativity, invigorating artists by offering new perspectives and tools for expression.

- **Flashback:** Historical comparisons highlight how past technological advances, like photography, initially threatened traditional art forms but eventuall